<a href="https://colab.research.google.com/github/ykitaguchi77/CongenitalGlaucoma_AI_project/blob/main/cornea_Yolact_segmentation_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Yolact_cornea_ellipse_segmentation**

参考：https://farml1.com/yolact：：

In [ ]:
"""
glaucoma_cornea_ellipse_coco/
├── annotations/
│   ├── instances_default.json  # COCO形式のアノテーションファイル
│
└── images/
    ├── 0-1.jpg  # 画像ファイル
    ├── 1-0.jpg
    ├── 2-1.jpg
    ├── ...

"""

In [18]:
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib
!pip install torchvision
!pip install torch


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pycocotools-2.0.6.tar.gz (24 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pycocotools


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [16 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-38
  creating build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\coco.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\cocoeval.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\mask.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\__init__.py -> build\lib.win-amd64-cpython-38\pycocotools
  running build_ext
  cythoning pycocotools/_mask.pyx to pycocotools\_mask.c
  building 'pycocotools._mask' extension
  C:\Users\ykita\AppData\Local\Temp\pip-build-env-cdzsu1wq\overlay\Lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\ykita\AppData\Local\Temp\pip-install-gxabxtik\pycocotools_6fe2d766bcbc4


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%cd "F:\先天性緑内障"
!git clone https://github.com/dbolya/yolact.git

F:\先天性緑内障


Cloning into 'yolact'...


#**学習のためフォルダを整理**

In [37]:
"""
# 画像をtrainとvalに分ける
"""

import os
import random
import shutil

# 画像フォルダと分割先のフォルダを指定
image_folder = "F:/先天性緑内障/glaucoma_cornea_ellipse_coco/images"
train_folder = "F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/images/train"
val_folder = "F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/images/val"

# フォルダを作成
if os.path.exists(train_folder):
    shutil.rmtree(train_folder) 
os.makedirs(train_folder, exist_ok=True)

if os.path.exists(val_folder):
    shutil.rmtree(val_folder) 
os.makedirs(val_folder, exist_ok=True)

# 画像フォルダ内の画像ファイルを取得
image_files = os.listdir(image_folder)
random.Random(1).shuffle(image_files)  # ランダムシードを1に設定してシャッフル

# 7:3に分割
train_size = int(len(image_files) * 0.7)
train_files = image_files[:train_size]
val_files = image_files[train_size:]

# 画像ファイルをコピー
for file in train_files:
    src = os.path.join(image_folder, file)
    dst = os.path.join(train_folder, file)
    shutil.copy(src, dst)

for file in val_files:
    src = os.path.join(image_folder, file)
    dst = os.path.join(val_folder, file)
    shutil.copy(src, dst)


In [36]:
dataset_path = f"F:\先天性緑内障\glaucoma_cornea_ellipse_coco"
cocojson_path = f"{dataset_path}/annotations/instances_default.json"
annotatedjson_path = f"{dataset_path}/annotations/annotated.json"

In [49]:
"""
# 画像のfile_nameに基づいてjsonを分割
"""

import json
import os

# トレーニングデータとバリデーションデータの画像フォルダのパス
train_image_dir = f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/images/train"
val_image_dir = f"F:/先天性緑内障\/glaucoma_cornea_ellipse_coco_train/images/val"

# COCO JSONファイルのパス
json_file_path = f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco/annotations/instances_default.json"

# 新しいJSONファイルの保存先
train_json_path = f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/annotations/train.json"
val_json_path = f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/annotations/val.json"

# COCO JSONファイルを読み込む
with open(json_file_path, "r") as f:
    coco_json = json.load(f)

# トレーニングデータとバリデーションデータに分割する
train_data = {"images": [], "annotations": [], "categories": coco_json["categories"]}
val_data = {"images": [], "annotations": [], "categories": coco_json["categories"]}

for image in coco_json["images"]:
    if os.path.exists(os.path.join(train_image_dir, image["file_name"])):
        train_data["images"].append(image)
    elif os.path.exists(os.path.join(val_image_dir, image["file_name"])):
        val_data["images"].append(image)

for annotation in coco_json["annotations"]:
    image_id = annotation["image_id"]
    if image_id in [image["id"] for image in train_data["images"]]:
        train_data["annotations"].append(annotation)
    elif image_id in [image["id"] for image in val_data["images"]]:
        val_data["annotations"].append(annotation)

# トレーニングデータとバリデーションデータのJSONファイルを保存する
with open(train_json_path, "w") as f:
    json.dump(train_data, f)

with open(val_json_path, "w") as f:
    json.dump(val_data, f)


In [53]:
# https://farml1.com/yolact/
# https://qiita.com/PoodleMaster/items/70cf03c1750fb58debb1

# yolact > data > config.pyの --DATASETS--の最後に以下を追加

"""
my_custom_dataset = dataset_base.copy({
    'name': 'My Dataset',

    'train_images': f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/images/train",
    'train_info':   f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/annotations/train.json"

    'valid_images': f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/images/val",
    'valid_info':   f"F:/先天性緑内障/glaucoma_cornea_ellipse_coco_train/annotations/val.json",

    'has_gt': True,

    # jsonの[categories]-[name]を参照して設定すること
    'class_names': ('corneal margin')
})


"""


#yolact > data > config.pyの --YOLACT v1.0 CONFIGS-- を以下に示すようにcoco2017をmy_customへ書き換え

"""
    # Dataset stuff
    'dataset': coco2017_dataset,
    'num_classes': len(coco2017_dataset.class_names) + 1,
    ↓
    # Dataset stuff
    'dataset': my_custom_dataset,
    'num_classes': len(my_custom_dataset.class_names) + 1,
"""

"\n    # Dataset stuff\n    'dataset': coco2017_dataset,\n    'num_classes': len(coco2017_dataset.class_names) + 1,\n    ↓\n    # Dataset stuff\n    'dataset': my_custom_dataset,\n    'num_classes': len(my_custom_dataset.class_names) + 1,\n"

In [ ]:
"""
YOLACTのGithubの中段Evaluationの表にある「yolact_base_54_800000.pth」をダウンロード
ダウンロードした事前学習済みデータは、yolact/weightsディレクトリ下へ格納
"""

In [62]:
yolact_path = f"F:/先天性緑内障/yolact"

!cd $yolact_path #ご自身の環境で作業ディレクトリへ移動してください。
!python train.py --config yolact_base_config --resume weights/yolact_base_54_800000.pth --start_iter 0 --batch_size 8 --num_workers 0 --validation_epoch -1

指定されたパスが見つかりません。
Traceback (most recent call last):
  File "train.py", line 1, in <module>
    from data import *
  File "F:\先天性緑内障\yolact\data\__init__.py", line 2, in <module>
    from .coco import *
  File "F:\先天性緑内障\yolact\data\coco.py", line 10, in <module>
    from pycocotools import mask as maskUtils
ModuleNotFoundError: No module named 'pycocotools'


In [67]:
!pip install cython
!pip install git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI




[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/waleedka/coco.git to c:\users\ykita\appdata\local\temp\pip-req-build-i2xvc_vy
  Resolved https://github.com/waleedka/coco.git to commit f83e9552d4ca57e15a16dca2efe3dcd80693358a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pycocotools
Failed to build pycocotools
  Running setup.py install for pycocotools: started
  Running setup.py install for pycocotools: finished with status 'error'


  Running command git clone --filter=blob:none --quiet https://github.com/waleedka/coco.git 'C:\Users\ykita\AppData\Local\Temp\pip-req-build-i2xvc_vy'
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [13 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-38
  creating build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\coco.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\cocoeval.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\mask.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\__init__.py -> build\lib.win-amd64-cpython-38\pycocotools
  running build_ext
  building 'pycocotools._mask' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of

In [65]:
!pip install wheel setuptools pip --upgrade
!pip3 install wheel setuptools pip --upgrade

  Using cached setuptools-67.3.3-py3-none-any.whl (1.1 MB)
     ---------------------------------------- 2.1/2.1 MB 7.7 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\ykita\AppData\Local\Programs\Python\Python38\python.exe -m pip install wheel setuptools pip --upgrade

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached setuptools-67.3.3-py3-none-any.whl (1.1 MB)
  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\ykita\AppData\Local\Programs\Python\Python38\python.exe -m pip install wheel setuptools pip --upgrade

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
!pip install numpy --use-deprecated=legacy-resolver
!pip3 install numpy --use-deprecated=legacy-resolver


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
